<a href="https://colab.research.google.com/github/deek2689/CERC_AI/blob/main/GPT2_Finetune_Safecity_commenting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install datasets -q

In [14]:
!pip install accelerate transformers torch trl -q -U

In [15]:
import pandas as pd
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, AdamW
from tqdm import tqdm
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, classification_report


In [16]:

train_path = 'https://raw.githubusercontent.com/deek2689/CERC_AI/refs/heads/main/SafeCity%20Datasets/train.csv'
test_path = 'https://raw.githubusercontent.com/deek2689/CERC_AI/refs/heads/main/SafeCity%20Datasets/test.csv'
val_path = 'https://raw.githubusercontent.com/deek2689/CERC_AI/refs/heads/main/SafeCity%20Datasets/dev.csv'

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
val_data = pd.read_csv(val_path)

In [17]:
train_data.head()

,Description,Category
0,"Was walking along crowded street, holding mums...",0
1,This incident took place in the evening.I was ...,0
2,I WAS WAITING FOR THE BUS. A MAN CAME ON A BIK...,1
3,Incident happened inside the train,0
4,I witnessed an incident when a chain was bruta...,0


In [18]:
# Define the instruction text
instruction = "Classify if the following statement falls under commenting related to sexual harassment. The output must be a single label: 'True' or 'False'."

def format_dataset(row):
    """
    Formats the dataset into the required structure for GPT-2 training.
    """
    formatted_text = (
        f"### Instruction:\n{instruction}\n\n"
        f"### Input:\n{row['Description']}\n\n"
        f"### Response:\n" ## Content after Response is kept empty since labels are passed separately
    )
    label = 1 if row['Category'] == 1 else 0  # Convert category to binary
    return formatted_text, label


In [19]:
def process_dataset(df):
    formatted_texts = []
    labels = []
    for _, row in df.iterrows():
        formatted_text, label = format_dataset(row)
        formatted_texts.append(formatted_text)
        labels.append(label)
    return formatted_texts, labels

formatted_texts_train, labels_train = process_dataset(train_data)
formatted_texts_val, labels_val = process_dataset(val_data)
formatted_texts_test, labels_test = process_dataset(test_data)


In [20]:
formatted_texts_train[1]

"### Instruction:\nClassify if the following statement falls under commenting related to sexual harassment. The output must be a single label: 'True' or 'False'.\n\n### Input:\nThis incident took place in the evening.I was in the metro when two guys started staring.\n\n### Response:\n"

In [21]:
# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Set the padding token
tokenizer.pad_token = tokenizer.bos_token
tokenizer.padding_side = "left"

# Tokenize the datasets
def tokenize_dataset(formatted_texts, labels, max_length=512):
    tokenized = tokenizer(
        formatted_texts, padding=True, truncation=True, max_length=max_length, return_tensors="pt"
    )
    input_ids = tokenized['input_ids']
    attention_mask = tokenized['attention_mask']
    labels_tensor = torch.tensor(labels)
    return TensorDataset(input_ids, attention_mask, labels_tensor)

train_dataset = tokenize_dataset(formatted_texts_train, labels_train)
val_dataset = tokenize_dataset(formatted_texts_val, labels_val)
test_dataset = tokenize_dataset(formatted_texts_test, labels_test)


In [22]:
train_dataset[1]

(tensor([50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 5

In [23]:
batch_size = 8

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)


In [24]:
# Load GPT-2 model with a classification head
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)
model.config.pad_token_id = tokenizer.pad_token_id
model.to('cuda')

# Define optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [25]:
# Training and validation loop
model.train()
num_epochs = 3
for epoch in tqdm(range(num_epochs), desc="Epochs"):
    print(f"Epoch {epoch + 1} of {num_epochs}")

    # Training
    total_train_loss = 0
    for batch in tqdm(train_dataloader, desc="Training"):
        input_ids, attention_masks, batch_labels = batch
        input_ids, attention_masks, batch_labels = input_ids.to('cuda'), attention_masks.to('cuda'), batch_labels.to('cuda')
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_masks, labels=batch_labels)
        loss = outputs.loss
        total_train_loss += loss.item()
        loss.backward()
        optimizer.step()

    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"Training loss: {avg_train_loss}")

    # Validation
    model.eval()
    val_predictions = []
    val_true_labels = []
    total_val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc="Validation"):
            input_ids, attention_masks, batch_labels = batch
            input_ids, attention_masks, batch_labels = input_ids.to('cuda'), attention_masks.to('cuda'), batch_labels.to('cuda')
            outputs = model(input_ids=input_ids, attention_mask=attention_masks, labels=batch_labels)
            loss = outputs.loss
            total_val_loss += loss.item()

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)
            pred_classes = torch.argmax(probs, dim=1)

            val_predictions.extend(pred_classes.cpu().numpy())
            val_true_labels.extend(batch_labels.cpu().numpy())

    avg_val_loss = total_val_loss / len(val_dataloader)
    val_macro_f1 = f1_score(val_true_labels, val_predictions, average='macro')
    print(f"Validation loss: {avg_val_loss}")
    print(f"Validation Macro F1 score: {val_macro_f1}")

    # Set the model back to training mode
    model.train()


Epochs:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1 of 3



Training: 100%|██████████| 901/901 [12:16<00:00,  1.22it/s]


Training loss: 0.7029349109804988



Epochs:  33%|███▎      | 1/3 [12:36<25:13, 756.50s/it]

...

Validation loss: 0.47625487658285326
Validation Macro F1 score: 0.7463160225914284
Epoch 3 of 3



Training: 100%|██████████| 901/901 [12:16<00:00,  1.22it/s]


Training loss: 0.4700377027914507



Epochs: 100%|██████████| 3/3 [37:50<00:00, 756.86s/it]

Validation loss: 0.4532312283352498
Validation Macro F1 score: 0.7734594837078423


In [26]:
# Testing loop
test_predictions = []
test_true_labels = []
model.eval()
with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Testing"):
        input_ids, attention_masks, batch_labels = batch
        input_ids, attention_masks, batch_labels = input_ids.to('cuda'), attention_masks.to('cuda'), batch_labels.to('cuda')
        outputs = model(input_ids=input_ids, attention_mask=attention_masks, labels=batch_labels)

        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=1)
        pred_classes = torch.argmax(probs, dim=1)

        test_predictions.extend(pred_classes.cpu().numpy())
        test_true_labels.extend(batch_labels.cpu().numpy())

test_macro_f1 = f1_score(test_true_labels, test_predictions, average='macro')
test_F1 = f1_score(test_true_labels, test_predictions)
test_accuracy = accuracy_score(test_true_labels, test_predictions)
test_precision= precision_score(test_true_labels, test_predictions)
test_recall= recall_score(test_true_labels, test_predictions)
print(f"Test Macro F1 score: {test_macro_f1}")
print(f"Test Regular F1 score: {test_F1}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Recall: {test_recall}")


Testing: 100%|██████████| 213/213 [00:54<00:00,  3.91it/s]

Test Macro F1 score: 0.7770763344382399
Test Regular F1 score: 0.6951672862453532
Test Accuracy: 0.807172251616696
Test Recall: 0.5718654434250765


In [27]:
from sklearn.metrics import classification_report

# Print the classification report
print(classification_report(test_true_labels, test_predictions))

              precision    recall  f1-score   support

           0       0.78      0.95      0.86      1047
           1       0.89      0.57      0.70       654

    accuracy                           0.81      1701
   macro avg       0.83      0.76      0.78      1701
weighted avg       0.82      0.81      0.80      1701

